In [ ]:
# Dependencies
import requests
import json
import os
from dotenv import load_dotenv
import pprint
import pandas as pd

# Load environment variables from .env file
load_dotenv()

# Access the API key
api_key = os.getenv("API_KEY")

In [ ]:
# place the url
url = "https://v6.exchangerate-api.com/v6/"

# Specify the start date
start_date = "2008/1/1"

# Specify the end date
end_date = "2008/12/31"

# Construct the url
response = requests.get(url + api_key + "history/AUD/" + start_date)

response

In [ ]:
# Construct the 2nd url
response2 = requests.get(url + api_key + "history/AUD/" + end_date)

response2

In [ ]:
# Get the data into a JSON format
data1 = response.json()
data2 = response2.json()

# Convert the dictionary to a dataframe
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# Make the index a column
df1_reset = df1.reset_index()
df2_reset = df2.reset_index()
# Move the 'index' column to a specific position (8 is the column count)
df1_reset.insert(8, 'QuoteCurrency', df1_reset['index'])
df2_reset.insert(8, 'QuoteCurrency', df2_reset['index'])

# Drop the old 'index' column
df1_clean = df1_reset.drop(columns=['index', 'result', 'documentation', 'terms_of_use'])
df2_clean = df2_reset.drop(columns=['index', 'result', 'documentation', 'terms_of_use'])

# Concatenate DataFrames along rows
combined_df = pd.concat([df1_clean, df2_clean], ignore_index=True)

combined_df

In [ ]:
# Filter the dataset for the start and end of the year
start_of_year = combined_df[(combined_df['month'] == 1) & (combined_df['day'] == 1)]
end_of_year = combined_df[(combined_df['month'] == 12) & (combined_df['day'] == 31)]

# Merge the start and end of the year datasets on the relevant columns
year_end_analysis = pd.merge(start_of_year, end_of_year, on=['year', 'base_code', 'QuoteCurrency'], suffixes=('_start', '_end'))

# Calculate the percentage change in conversion rates
year_end_analysis['percentage_change'] = ((year_end_analysis['conversion_rates_end'] - year_end_analysis['conversion_rates_start']) / year_end_analysis['conversion_rates_start']) * 100

# Display the year-end analysis per currency pair
year_end_analysis